In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import pylab
import glob, os
import scipy.stats as stats
from scipy.stats import gaussian_kde
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import timeit
%matplotlib inline

In [2]:
# Format to remove scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# options of samples

In [3]:
Sample = pd.read_csv('R:/Angela/fast_trips/2015SampleDataSpring_6day.csv')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2868: DtypeWarning: Columns (72,75,77,83,117) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
len(np.unique(Sample.QSTOP))

3179

In [51]:
len(np.unique(Sample.VEHNO))

284

#my_data

In [5]:
my_data = Sample

In [52]:
def data_content(data):
    data = data[['ON', 'OFF', 'VEHNO', 'ANAME', 'QSTOP','STOPA', 'YR', 'HR', 'MIN', 'SEC', 'DHR', 'DMIN', 'DSEC', 'ROUTE', 'LOAD', 'date_id']]
    return data

In [53]:
test = data_content(my_data)
test.shape

(494613, 16)

#Prepare bus info

In [55]:
vehicles = pd.read_csv(r'R:\Angela\fast_trips\Vehicles.csv')
fleet = pd.read_csv(r'R:\Angela\fast_trips\Copy of Fleet.csv')

# Artic
vehicles.Artic = vehicles.Length.map({"60'" : 1, "40'" : 0, "30'" : 0})
vehicles.loc[:,'Artic'] = pd.Series(vehicles.Artic, index=vehicles.index)
df_artic = vehicles.set_index('Equip_Type').to_dict()['Artic']
fleet['Artic'] = fleet['Equip_Type'].map(df_artic)
df_vehnum_artic = fleet.set_index('VehNum').to_dict()['Artic']

# Low Floor
vehicles.Floor = vehicles['Low Floor'].map({'Y': 1, 'N' : 0})
vehicles.loc[:,'Floor'] = pd.Series(vehicles.Floor, index=vehicles.index)
df_floor = vehicles.set_index('Equip_Type').to_dict()['Floor']
fleet['Floor'] = fleet['Equip_Type'].map(df_floor)
df_vehnum_floor = fleet.set_index('VehNum').to_dict()['Floor']

# Door
df_doors = vehicles.set_index('Equip_Type').to_dict()['Doors']
fleet['Doors'] = fleet['Equip_Type'].map(df_doors)
df_vehnum_doors = fleet.set_index('VehNum').to_dict()['Doors']

# Capacity
vehicles.loc[:,'Total Capacity'] = pd.Series(vehicles['Total Capacity'], index=vehicles.index)
df_capacity = vehicles.set_index('Equip_Type').to_dict()['Total Capacity']
fleet['capacity'] = fleet['Equip_Type'].map(df_capacity)
df_vehnum_capacity = fleet.set_index('VehNum').to_dict()['capacity']

# Prepare route type

In [56]:
route_type = pd.read_csv(r'R:\Angela\fast_trips\MuniRouteTypes.csv')
route_type = route_type.dropna()
dict_route_type = {}
dict_route_type = route_type.set_index('APC Route ID')['Type'].to_dict()

#Step1: Prepare basic variables

In [57]:
def get_x_y(data):
    start = timeit.default_timer()
    # Before cleaning records, get the load data from the previous stop
    data['pre_load'] = data['LOAD'].shift()

    # Get rid of rows where certain fields has null/nan values
    data = data.dropna(subset = ['ON', 'OFF', 'VEHNO'])
    data = data[data['ON'] + data['OFF'] != 0]

    # COMPUTE TIMESTOP=((HR * 3600) + (MIN * 60) + SEC)
    start = timeit.default_timer()
    data['COMPUTE_TIMESTOP'] = data['HR']*3600 + data['MIN']*60 + data['SEC']
    # COMPUTE DOORCLOSE=(( DHR * 3600) + (DMIN * 60) + DSEC)
    data['COMPUTE_DOORCOLSE'] = data['DHR']*3600 + data['DMIN']*60 + data['DSEC']
    # COMPUTE DOORDWELL=DOORCLOSE - TIMESTOP
    data['COMPUTE_DOORDWELL'] = data['COMPUTE_DOORCOLSE'] - data['COMPUTE_TIMESTOP']
    # Appling door dwell time less than 120 secs
    data = data.loc[data['COMPUTE_DOORDWELL'] <= 120]
    data = data[data['COMPUTE_DOORDWELL'] != 0]
    stop = timeit.default_timer()
    print 'compute dwell time:', stop - start

    # Keep rows that satisfy a query:
    start = timeit.default_timer()
    data['Doors'] = data['VEHNO'].map(df_vehnum_doors) 
    data['Artic'] = data['VEHNO'].map(df_vehnum_artic)
    data['Floor'] = data['VEHNO'].map(df_vehnum_floor)
    data['capacity'] = data['VEHNO'].map(df_vehnum_capacity)
    data['two_doors'] = data['Doors'].map({2: 1, 3: 0})
    data['three_doors'] = data['Doors'].map({2: 0, 3: 1})
    
    # Create dummie variables for route id
    data['Route Type'] = data['ROUTE'].map(dict_route_type)
    just_dummies_route = pd.get_dummies(data['Route Type'])
    step_1 = pd.concat([data, just_dummies_route], axis=1)
    step_1.drop(['Local'], inplace=True, axis=1)
    data = step_1
    stop = timeit.default_timer()
    print 'add veh&route info:', stop - start

    # Create interaction variables
    start = timeit.default_timer()
    data['on_threedoors'] = data['ON']*data['three_doors']
    data['off_threedoors'] = data['OFF']*data['three_doors']
    data['on_floor'] = data['ON']*data['Floor']
    data['off_floor'] = data['OFF']*data['Floor']
    data['floor_threedoors'] = data['Floor']*data['three_doors']
    data['floor_twodoors'] = data['Floor']*data['two_doors']
    data['on_express'] = data['ON']*data['Express']
    data['off_express'] = data['OFF']*data['Express']
    data['on_rapid'] = data['ON']*data['Rapid']
    data['off_rapid'] = data['OFF']*data['Rapid']
    data['on_owl'] = data['ON']*data['OWL']
    data['off_owl'] = data['OFF']*data['OWL']
    stop = timeit.default_timer()
    print 'add interaction variables:', stop - start

    return data

## Step2: Adding more passenger activity variables

In [58]:
def passenger_var(data):
    start = timeit.default_timer()
    data['max_pasg'] = data[['ON', 'OFF']].max(axis=1)
    print 'data shape:', data.shape
    data['abs_pasg'] = (data['ON'] - data['OFF']).abs()
    print 'data shape:', data.shape
    
    # A passenger friction factor was constructed to account for passenger activity on buses with standees. 
    # It was posited that heavily loaded buses have greater dwell times. 
    # STANDEES are the number of passengers when LOAD minus 60% of bus capacity is positive. 
    data['pre_standees']= data['pre_load'] - 0.60 * data['capacity']
    data['pre_crowding']= data.apply(lambda x: x['pre_standees'] > 0, axis=1).map({False: 0, True: 1})
    # A proxy variable was constructed by adding ONS, OFFS, and STANDEES.
    data['friction'] = ((data['ON'] + data['OFF'] + (data['pre_standees']).abs()) * data['pre_crowding']).abs()
    print 'data shape:', data.shape
    stop = timeit.default_timer()
    print 'add passenger activity variables:', stop - start

    # Remove the corner data, which is the first and last stop data
    start = timeit.default_timer()
    data['eol'] = data.apply(lambda x: '- EOL' in x['ANAME'], axis=1).map({False: 1, True: 0})
    # Remove the last stop
    data = data.loc[data['eol'] == 1]
    # Remove the first stop
    data = data.loc[data['STOPA'] != 1]
    stop = timeit.default_timer()
    print 'remove corner data:', stop - start
    
    return data

In [59]:
step1 = get_x_y(test)

compute dwell time: 0.139913834817
add veh&route info: 0.342111339811
add interaction variables: 0.0289090743863


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [60]:
step2 = passenger_var(step1)

data shape: (272383, 43)
data shape: (272383, 44)
data shape: (272383, 47)
add passenger activity variables: 4.05109511525
remove corner data: 4.09343878653


In [74]:
data = step2

In [75]:
# Create vehicleID & date column
data['vehno_date'] = data.VEHNO.astype(str) + '_' + data.date_id.astype(str)

# Create training (70%) and validating (30%) dataset 

In [76]:
msk = np.random.rand(len(data)) < 0.7
df_train = data[msk]
df_test = data[~msk]

In [77]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [78]:
print df_train.shape
print df_test.shape

(188623, 49)
(80984, 49)


In [79]:
df_train.columns

Index([u'ON', u'OFF', u'VEHNO', u'ANAME', u'QSTOP', u'STOPA', u'YR', u'HR',
       u'MIN', u'SEC', u'DHR', u'DMIN', u'DSEC', u'ROUTE', u'LOAD', u'date_id',
       u'pre_load', u'COMPUTE_TIMESTOP', u'COMPUTE_DOORCOLSE',
       u'COMPUTE_DOORDWELL', u'Doors', u'Artic', u'Floor', u'capacity',
       u'two_doors', u'three_doors', u'Route Type', u'Express', u'OWL',
       u'Rapid', u'on_threedoors', u'off_threedoors', u'on_floor',
       u'off_floor', u'floor_threedoors', u'floor_twodoors', u'on_express',
       u'off_express', u'on_rapid', u'off_rapid', u'on_owl', u'off_owl',
       u'max_pasg', u'abs_pasg', u'pre_standees', u'pre_crowding', u'friction',
       u'eol', u'vehno_date'],
      dtype='object')

Note: Passenger activity variables: on, off, friction

# 1. Linear regression model with basic vairables

In [83]:
lm3 = smf.ols(formula = 'COMPUTE_DOORDWELL ~ ON + OFF + Floor+ three_doors + Express + OWL + Rapid', data = df_train).fit()
print lm3.summary()

                            OLS Regression Results                            
Dep. Variable:      COMPUTE_DOORDWELL   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     8868.
Date:                Thu, 17 Mar 2016   Prob (F-statistic):               0.00
Time:                        15:34:35   Log-Likelihood:            -7.7312e+05
No. Observations:              188623   AIC:                         1.546e+06
Df Residuals:                  188615   BIC:                         1.546e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      11.8906      0.059    201.333      

# 2. Mixed linear regression model:  based on Stop ID

In [84]:
ind = sm.cov_struct.Exchangeable()
fml = "COMPUTE_DOORDWELL ~ ON + OFF + Floor + three_doors + Express + OWL + Rapid"
mod_gee = GEE.from_formula(fml, groups=df_train["QSTOP"], cov_struct=ind, data=df_train)
mod_gee = mod_gee.fit()
print mod_gee.summary()
print mod_gee.cov_struct.summary()

                               GEE Regression Results                              
Dep. Variable:           COMPUTE_DOORDWELL   No. Observations:               188623
Model:                                 GEE   No. clusters:                     3013
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                 590
Family:                           Gaussian   Mean cluster size:                62.6
Dependence structure:         Exchangeable   Num. iterations:                    11
Date:                     Thu, 17 Mar 2016   Scale:                         215.515
Covariance type:                    robust   Time:                         15:34:46
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      11.2439      0.170     66.297      0.000        10.911    11.576
ON  

In [85]:
mod_lme = MixedLM.from_formula(fml, groups=df_train["QSTOP"], data=df_train)
mod_lme = mod_lme.fit()
print mod_lme.summary()

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: COMPUTE_DOORDWELL
No. Observations: 188623  Method:             REML             
No. Groups:       3013    Scale:              193.3223         
Min. group size:  1       Likelihood:         -766371.2657     
Max. group size:  590     Converged:          Yes              
Mean group size:  62.6                                         
----------------------------------------------------------------
                Coef.   Std.Err.     z     P>|z|  [0.025  0.975]
----------------------------------------------------------------
Intercept       11.327     0.116   97.869  0.000  11.100  11.554
ON               1.893     0.012  153.704  0.000   1.869   1.917
OFF              1.185     0.013   94.608  0.000   1.160   1.209
Floor           -3.168     0.119  -26.618  0.000  -3.401  -2.935
three_doors     -1.672     0.123  -13.563  0.000  -1.914  -1.430
Express         -1.731     0.199   -8.705  0.

C:\Anaconda\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


# 3. Mixed linear regression model: Based on Vehicle ID

In [86]:
ind = sm.cov_struct.Exchangeable()
fml = "COMPUTE_DOORDWELL ~ ON + OFF + Floor + three_doors + Express + OWL + Rapid"
mod_gee = GEE.from_formula(fml, groups=df_train["vehno_date"], cov_struct=ind, data=df_train)
mod_gee = mod_gee.fit()
print mod_gee.summary()
print mod_gee.cov_struct.summary()

                               GEE Regression Results                              
Dep. Variable:           COMPUTE_DOORDWELL   No. Observations:               188623
Model:                                 GEE   No. clusters:                      940
Method:                        Generalized   Min. cluster size:                   3
                      Estimating Equations   Max. cluster size:                 499
Family:                           Gaussian   Mean cluster size:               200.7
Dependence structure:         Exchangeable   Num. iterations:                     7
Date:                     Thu, 17 Mar 2016   Scale:                         212.696
Covariance type:                    robust   Time:                         15:37:49
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      11.8753      0.152     78.189      0.000        11.578    12.173
ON  

In [87]:
mod_lme = MixedLM.from_formula(fml, groups=df_train["vehno_date"], data=df_train)
mod_lme = mod_lme.fit()
print mod_lme.summary()

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: COMPUTE_DOORDWELL
No. Observations: 188623  Method:             REML             
No. Groups:       940     Scale:              207.6254         
Min. group size:  3       Likelihood:         -771680.2026     
Max. group size:  499     Converged:          Yes              
Mean group size:  200.7                                        
----------------------------------------------------------------
                Coef.   Std.Err.     z     P>|z|  [0.025  0.975]
----------------------------------------------------------------
Intercept       11.827     0.135   87.656  0.000  11.562  12.091
ON               2.056     0.011  194.182  0.000   2.035   2.077
OFF              1.341     0.011  124.276  0.000   1.320   1.362
Floor           -3.600     0.206  -17.479  0.000  -4.004  -3.196
three_doors     -1.522     0.229   -6.658  0.000  -1.970  -1.074
Express         -1.915     0.246   -7.790  0.

# Correlation table

find the top correlations in a correlation matrix with Pandas

In [98]:
c = X_train.corr().abs()
s = c.unstack()
so = s.order(kind="quicksort")

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: FutureWarning: order is deprecated, use sort_values(...)
  app.launch_new_instance()


In [99]:
print len(so)
soo = so.dropna()
soo = soo[soo >= 0.500]
soo = soo[soo != 1.0]
print len(soo)

361
28


In [100]:
soo

on_floor        Floor            0.504
Floor           on_floor         0.504
                off_floor        0.506
off_floor       Floor            0.506
off_express     Express          0.517
Express         off_express      0.517
three_doors     on_threedoors    0.538
on_threedoors   three_doors      0.538
abs_pasg        ON               0.544
ON              abs_pasg         0.544
abs_pasg        OFF              0.553
OFF             abs_pasg         0.553
off_threedoors  three_doors      0.557
three_doors     off_threedoors   0.557
on_express      Express          0.560
Express         on_express       0.560
off_threedoors  OFF              0.570
OFF             off_threedoors   0.570
on_threedoors   ON               0.578
ON              on_threedoors    0.578
OWL             off_owl          0.590
off_owl         OWL              0.590
on_rapid        Rapid            0.616
Rapid           on_rapid         0.616
off_rapid       Rapid            0.633
Rapid           off_rapid

In [101]:
print 'end'

end
